# Question 1: Capturing the Wikipedia Table

In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=85ed4ad7262e2f725df4a731f68401e55c18bd26f38d4c19738d51100607635f
  Stored in directory: c:\users\srirama\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [3]:
!pip install requests

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [12]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [13]:
soup = BeautifulSoup(html, 'html.parser')

In [79]:
Toronto_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
for row in soup.find("tbody").find_all("tr"):
    col = (row.find_all("td"))
    if(len(col)>0):
        pc = col[0].text.rstrip('\n')
        bh = col[1].text.rstrip('\n')
        nd = col[2].text.rstrip('\n')
        
        Toronto_df = Toronto_df.append({"PostalCode":pc,"Borough":bh,"Neighborhood":nd  }, ignore_index=True)
        
Toronto_df = Toronto_df[Toronto_df['Borough']!='Not assigned'].reset_index(drop=True)

In [80]:
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [85]:
toronto_grouped = Toronto_df.groupby(['PostalCode', 'Borough'], as_index=False)
toronto_concat = toronto_grouped.agg(lambda x: ", ".join(x))
toronto_concat.head(30)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [90]:
toronto_concat.shape

(103, 3)

# Question 2: Attach lat/long to each postalcode